In [ ]:
max_ch_disp = 10  # max channels to initially display
max_t_disp = 5 # max time in seconds to initially display

spacing = 2.5  # Spacing between channels
offset = np.std(data) * spacing

# Create a hv.Curve element per channel
channel_curves = []
max_data = data.max()
hover = HoverTool(tooltips=[
    ("Channel", "@channel"),
    ("Time", "$x s"),
    ("Amplitude", "@original_amplitude µV")])
for i, channel_data in enumerate(data):
    offset_data = channel_data + (i * offset)
    max_data = max(offset_data.max(), max_data)  # update max
    ds = Dataset((time, offset_data, channel_data, channels[i]), ["Time", "Amplitude", "original_amplitude", "channel"])
    channel_curves.append(
        hv.Curve(ds, "Time", ["Amplitude", "original_amplitude", "channel"]).opts(
            color="black", line_width=1,
            tools=[hover, 'xwheel_zoom'], shared_axes=False))

# Create mapping from yaxis location to ytick for each channel
yticks = [(i * offset, ich) for i, ich in enumerate(channels)]

# Create an overlay of curves
eeg_viewer = hv.Overlay(channel_curves, kdims="Channel").opts(
    padding=0, xlabel="Time (s)", ylabel="Channel", yticks=yticks, show_legend=False, aspect=1.5, responsive=True,
    shared_axes=False, backend_opts={
        "x_range.bounds": (time.min(), time.max()),
        "y_range.bounds": (data.min(), max_data)})

# Get the y positions of the yticks to use as yaxis of minimap image
y_positions, _ = zip(*yticks)

# Compute z-scores across time for each channel
z_data = zscore(data, axis=1)

# Generate the zscored image for the minimap using the y tick positions from the eeg_viewer
minimap = rasterize(hv.Image((time, y_positions, z_data), ["Time (s)", "Channel"], "Amplitude (uV)"))
minimap = minimap.opts(
    cmap="RdBu_r", colorbar=False, xlabel='', alpha=.5, yticks=[yticks[0], yticks[-1]],
    height=100, responsive=True, default_tools=[''], shared_axes=False, clim=(-z_data.std()*2.5, z_data.std()*2.5))

# Create RangeToolLink between the minimap and the main EEG viewer 
if len(channels) < max_ch_disp:
    max_ch_disp = len(channels)
max_y_disp = np.max(data[max_ch_disp-1,:] + ((max_ch_disp-1) * offset))

time_s = len(time)/raw.info['sfreq']
if time_s < max_t_disp:
    max_t_disp = time_s
    
RangeToolLink(minimap, list(eeg_viewer.values())[0], axes=["x", "y"],
              boundsx=(None, max_t_disp),
              boundsy=(None, max_y_disp))

eeg_app = pn.Column((eeg_viewer + minimap).cols(1), min_height=650) #.servable(target='main')
eeg_app